# Complete Dataset Analysis (Clean)
## Hebrew Idiom Identification Dataset - Hebrew-Idioms-4800

Purpose: Comprehensive validation, statistics, and visualization of the dataset for professor review.

## Table of Contents
1. Setup and Data Loading
2. Basic Statistics
3. Data Quality Validation
4. Preprocessing & Span Verification
5. Label Distribution Analysis
6. Sentence & Idiom Length Analysis
7. Idiom Position Analysis
8. Polysemy Analysis
9. Lexical Diversity & Richness
10. Structural Complexity
11. Morphological Variance (Hebrew-Specific)
12. IOB2 Tag Validation
13. Sentence Type Analysis
14. Summary & Validation Report


In [66]:
# 1. Setup and Data Loading
import pandas as pd
import numpy as np
from collections import Counter
from pathlib import Path
import ast
import re

def parse_list(val):
    if isinstance(val, list):
        return val
    try:
        return ast.literal_eval(val)
    except Exception:
        return str(val).split()

DATA_PATH = Path('data/expressions_data_tagged.csv')
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}")

df = pd.read_csv(DATA_PATH, encoding='utf-8-sig')
df['tokens'] = df['tokens'].apply(parse_list)
df['iob_tags'] = df['iob_tags'].apply(parse_list)
for col in ['label','start_token','end_token','num_tokens','start_char','end_char']:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(int)

print('✅ Dataset loaded')
print(f"Shape: {len(df)} rows x {df.shape[1]} columns")
print(f"Columns: {list(df.columns)}")


✅ Dataset loaded
Shape: 4800 rows x 17 columns
Columns: ['id', 'sentence', 'base_pie', 'pie_span', 'label', 'label_str', 'tokens', 'iob_tags', 'start_token', 'end_token', 'num_tokens', 'char_mask', 'start_char', 'end_char', 'split', 'language', 'source']


## 2. Basic Statistics

In [67]:

print('='*70)
print('BASIC DATASET STATISTICS')
print('='*70)
print(f"📊 Total sentences: {len(df)}")
print(f"📊 Unique idioms: {df['base_pie'].nunique()}")
print(f"📊 Samples per idiom: {len(df) // df['base_pie'].nunique()}")
expr_counts = df['base_pie'].value_counts()
print(f"📊 Expression Occurrence Statistics:")
print(f"  Min: {expr_counts.min()}")
print(f"  Max: {expr_counts.max()}")
print(f"  Mean: {expr_counts.mean():.2f}")
print(f"  Std: {expr_counts.std():.2f}")
print(f"📊 Split sizes: {df['split'].value_counts().to_dict()}")


BASIC DATASET STATISTICS
📊 Total sentences: 4800
📊 Unique idioms: 60
📊 Samples per idiom: 80
📊 Expression Occurrence Statistics:
  Min: 80
  Max: 80
  Mean: 80.00
  Std: 0.00
📊 Split sizes: {'train': 3456, 'unseen_idiom_test': 480, 'validation': 432, 'test': 432}


## 3. Data Quality Validation

In [68]:
print('='*70)
print('DATA QUALITY VALIDATION')
print('='*70)
validation_results = {}
missing = df.isnull().sum().sum()
validation_results['Missing values'] = missing == 0
print(f"1. Missing values: {missing}/{len(df)*len(df.columns)}", '✅' if missing == 0 else '❌')
hashable_cols = [c for c in df.columns if c not in ['tokens','iob_tags']]
duplicates = df[hashable_cols].duplicated().sum()
validation_results['Duplicates'] = duplicates == 0
print(f"2. Duplicate rows: {duplicates}/{len(df)}", '✅' if duplicates == 0 else '❌')
id_valid = (df['id'].nunique() == len(df))
validation_results['ID sequence'] = id_valid
print('3. ID column validity (unique == total rows):', '✅' if id_valid else '❌')
label_mismatch = ((df['label_str'] == 'Literal') & (df['label'] != 0)).sum() + ((df['label_str'] == 'Figurative') & (df['label'] != 1)).sum()
validation_results['Label consistency'] = label_mismatch == 0
print(f"4. Label consistency errors: {label_mismatch}", '✅' if label_mismatch == 0 else '❌')
# Encoding checks
bom_count = df['sentence'].str.contains('﻿').sum()
zwsp_count = df['sentence'].str.contains('​').sum()
zwj_count = df['sentence'].str.contains('‍').sum()
lrm_count = df['sentence'].str.contains('‎').sum()
rlm_count = df['sentence'].str.contains('‏').sum()
encoding_clean = (bom_count + zwsp_count + zwj_count + lrm_count + rlm_count) == 0
validation_results['Encoding'] = encoding_clean
print(f"5. Encoding validation: BOM={bom_count}, ZWSP={zwsp_count}, ZWJ={zwj_count}, LRM={lrm_count}, RLM={rlm_count}", '✅' if encoding_clean else '❌')
# Hebrew text presence
hebrew_pattern = re.compile(r'[֐-׿]')
has_hebrew = df['sentence'].apply(lambda x: bool(hebrew_pattern.search(str(x)))).sum()
validation_results['Hebrew text'] = has_hebrew == len(df)
print(f"6. Sentences with Hebrew text: {has_hebrew}/{len(df)}", '✅' if has_hebrew == len(df) else '❌')
# Whitespace
trailing_ws = df['sentence'].str.endswith(' ').sum() + df['sentence'].str.startswith(' ').sum()
multi_spaces = df['sentence'].str.contains('  ').sum()
print("7. Whitespace issues:")
print(f"   Trailing/leading whitespace: {trailing_ws} ({100*trailing_ws/len(df):.2f}%)")
print(f"   Multiple consecutive spaces: {multi_spaces} ({100*multi_spaces/len(df):.2f}%)")


DATA QUALITY VALIDATION
1. Missing values: 0/81600 ✅
2. Duplicate rows: 0/4800 ✅
3. ID column validity (unique == total rows): ✅
4. Label consistency errors: 0 ✅
5. Encoding validation: BOM=0, ZWSP=0, ZWJ=0, LRM=0, RLM=0 ✅
6. Sentences with Hebrew text: 4800/4800 ✅
7. Whitespace issues:
   Trailing/leading whitespace: 0 (0.00%)
   Multiple consecutive spaces: 0 (0.00%)


## 4. Preprocessing & Span Verification

In [69]:

print('='*70)
print('SPAN & MASK VERIFICATION')
print('='*70)

char_span_errors = []
mask_errors = []
token_span_errors = []
iob_len_errors = []
valid_tags = {'O','B-IDIOM','I-IDIOM'}

for _, row in df.iterrows():
    sent = row['sentence']
    sc, ec = int(row['start_char']), int(row['end_char'])
    pie = row['pie_span']
    if sent[sc:ec] != pie:
        char_span_errors.append(row['id'])
    mask = str(row['char_mask'])
    if len(mask) != len(sent) or mask[sc:ec] != '1'*(ec-sc) or ('1' in mask[:sc] or '1' in mask[ec:]):
        mask_errors.append(row['id'])
    tokens = row['tokens']
    tags = row['iob_tags']
    if len(tokens) != len(tags) or len(tokens) != row['num_tokens']:
        iob_len_errors.append(row['id'])
    st, et = int(row['start_token']), int(row['end_token'])
    if not (0 <= st < et <= len(tokens)):
        token_span_errors.append(row['id'])
    elif ' '.join(tokens[st:et]) != pie:
        token_span_errors.append(row['id'])
    if any(t not in valid_tags for t in tags):
        iob_len_errors.append(row['id'])

print(f"Char span errors: {len(char_span_errors)}")
print(f"Char mask errors: {len(mask_errors)}")
print(f"Token span errors: {len(token_span_errors)}")
print(f"IOB length/tag errors: {len(iob_len_errors)}")


SPAN & MASK VERIFICATION
Char span errors: 0
Char mask errors: 0
Token span errors: 0
IOB length/tag errors: 0


In [70]:

print('='*70)
print('ADDITIONAL SPAN/IOB CONSISTENCY')
print('='*70)

span_mismatches = []
for _, row in df.iterrows():
    tags = row['iob_tags']
    spans = []
    current = None
    for i, t in enumerate(tags):
        if t == 'B-IDIOM':
            if current is not None:
                spans.append(current)
            current = [i, i+1]
        elif t == 'I-IDIOM' and current:
            current[1] = i+1
        else:
            if current is not None:
                spans.append(current)
                current = None
    if current is not None:
        spans.append(current)

    if len(spans) != 1:
        span_mismatches.append(row['id'])
        continue

    s, e = spans[0]
    if s != int(row['start_token']) or e != int(row['end_token']):
        span_mismatches.append(row['id'])
        continue
    tok_slice = ' '.join(row['tokens'][s:e])
    if tok_slice != row['pie_span']:
        span_mismatches.append(row['id'])

print(f"Span/IOB single-span mismatches: {len(span_mismatches)}")


ADDITIONAL SPAN/IOB CONSISTENCY
Span/IOB single-span mismatches: 0


In [71]:

print('='*70)
print('CROSS-FIELD SPAN CONSISTENCY')
print('='*70)

cf_errors = []
for _, row in df.iterrows():
    sent = row['sentence']
    sc, ec = int(row['start_char']), int(row['end_char'])
    pie = row['pie_span']
    tok_slice = ' '.join(row['tokens'][int(row['start_token']):int(row['end_token'])])
    if sent[sc:ec] != pie or tok_slice != pie:
        cf_errors.append(row['id'])
print(f"Cross-field span mismatches: {len(cf_errors)}")


CROSS-FIELD SPAN CONSISTENCY
Cross-field span mismatches: 0


In [72]:

print('='*70)
print('LABEL STRING CONSISTENCY')
print('='*70)

mismatch = df[~(((df['label']==0) & (df['label_str']=='Literal')) | ((df['label']==1) & (df['label_str']=='Figurative')))]
print(f"Label↔label_str mismatches: {len(mismatch)}")


LABEL STRING CONSISTENCY
Label↔label_str mismatches: 0


## 5. Label Distribution Analysis

In [73]:

print('='*70)
print('LABEL DISTRIBUTION')
print('='*70)
label_counts = df['label'].value_counts().sort_index()
print(label_counts)
print('Per split:')
print(df.groupby(['split','label'])['id'].count())


LABEL DISTRIBUTION
label
0    2400
1    2400
Name: count, dtype: int64
Per split:
split              label
test               0         216
                   1         216
train              0        1728
                   1        1728
unseen_idiom_test  0         240
                   1         240
validation         0         216
                   1         216
Name: id, dtype: int64


In [74]:

print('='*70)
print('SPLIT INTEGRITY CHECKS')
print('='*70)

# IDs should be unique across splits
id_split_counts = df.groupby('id')['split'].nunique()
leak_ids = id_split_counts[id_split_counts > 1].index.tolist()
print(f"IDs appearing in multiple splits: {len(leak_ids)}")

# Unseen idioms should not appear in seen splits
unseen_idioms = set(df[df['split']=='unseen_idiom_test']['base_pie'])
seen_idioms = set(df[df['split']!='unseen_idiom_test']['base_pie'])
leaking_unseen = unseen_idioms & seen_idioms
print(f"Unseen idioms leaking into seen splits: {len(leaking_unseen)}")


SPLIT INTEGRITY CHECKS
IDs appearing in multiple splits: 0
Unseen idioms leaking into seen splits: 0


## 6. Sentence & Idiom Length Analysis

In [75]:

print('='*70)
print('LENGTH ANALYSIS')
print('='*70)

sent_tokens = df['num_tokens']
sent_chars = df['sentence'].str.len()
idiom_tokens = df['end_token'] - df['start_token']
idiom_chars = df['pie_span'].str.len()

print('Sentence length (tokens):')
print(f"  Mean: {sent_tokens.mean():.2f} | Median: {sent_tokens.median():.0f} | Std: {sent_tokens.std():.2f} | Range: {sent_tokens.min()}-{sent_tokens.max()}")
print('Sentence length (chars):')
print(f"  Mean: {sent_chars.mean():.2f} | Median: {sent_chars.median():.0f} | Std: {sent_chars.std():.2f} | Range: {sent_chars.min()}-{sent_chars.max()}")
print('Idiom length (tokens):')
print(f"  Mean: {idiom_tokens.mean():.2f} | Median: {idiom_tokens.median():.0f} | Range: {idiom_tokens.min()}-{idiom_tokens.max()}")
print('Idiom length (chars):')
print(f"  Mean: {idiom_chars.mean():.2f} | Median: {idiom_chars.median():.0f} | Range: {idiom_chars.min()}-{idiom_chars.max()}")


LENGTH ANALYSIS
Sentence length (tokens):
  Mean: 17.47 | Median: 13 | Std: 9.11 | Range: 5-47
Sentence length (chars):
  Mean: 83.03 | Median: 63 | Std: 42.55 | Range: 22-193
Idiom length (tokens):
  Mean: 2.48 | Median: 2 | Range: 2-5
Idiom length (chars):
  Mean: 11.39 | Median: 11 | Range: 5-23


## 7. Idiom Position Analysis

In [76]:

print('='*70)
print('IDIOM POSITION ANALYSIS')
print('='*70)

df['position_ratio'] = df['start_token']/df['num_tokens']
cat = lambda r: 'start' if r<=0.33 else ('middle' if r<=0.67 else 'end')
df['position_category'] = df['position_ratio'].apply(cat)

print('Overall position counts:', df['position_category'].value_counts().to_dict())
print(f"Mean position ratio: {df['position_ratio'].mean():.4f}")
print('By label:')
print(df.groupby(['label','position_category'])['id'].count())
print('By split:')
print(df.groupby(['split','position_category'])['id'].count())


IDIOM POSITION ANALYSIS
Overall position counts: {'start': 3123, 'middle': 1449, 'end': 228}
Mean position ratio: 0.2678
By label:
label  position_category
0      end                    91
       middle                756
       start                1553
1      end                   137
       middle                693
       start                1570
Name: id, dtype: int64
By split:
split              position_category
test               end                    21
                   middle                126
                   start                 285
train              end                   170
                   middle               1041
                   start                2245
unseen_idiom_test  end                    14
                   middle                143
                   start                 323
validation         end                    23
                   middle                139
                   start                 270
Name: id, dtype: int64


## 8. Polysemy Analysis

In [77]:

print('='*70)
print('POLYSEMY ANALYSIS')
print('='*70)
expr_label = df.groupby(['base_pie','label'])['id'].count().unstack(fill_value=0)
polysemous = expr_label[(expr_label.get(0,0)>0) & (expr_label.get(1,0)>0)]
print(f"Polysemous idioms: {len(polysemous)}/{df['base_pie'].nunique()}")


POLYSEMY ANALYSIS
Polysemous idioms: 60/60


## 9. Lexical Diversity & Richness

In [78]:
print('='*70)
print('LEXICAL DIVERSITY')
print('='*70)
all_tokens = [t for row in df['tokens'] for t in row]
word_freq = Counter(all_tokens)
vocab = set(all_tokens)
print(f"Vocabulary size: {len(vocab):,}")
print(f"Total tokens: {len(all_tokens):,}")
print(f"Type-Token Ratio (TTR): {len(vocab)/len(all_tokens):.4f}")
hapax = sum(1 for v in word_freq.values() if v==1)
dis = sum(1 for v in word_freq.values() if v==2)
print(f"Hapax legomena: {hapax} ({hapax/len(vocab)*100:.1f}% of vocab)")
print(f"Dis legomena: {dis}")
print('Top 10 tokens:')
print(word_freq.most_common(10))
ctx=[]
for _, row in df.iterrows():
    toks=row['tokens']
    s=int(row['start_token']); e=int(row['end_token'])
    left=max(0,s-3); right=min(len(toks), e+3)
    ctx.extend(toks[left:s]); ctx.extend(toks[e:right])
ctx_freq=Counter(ctx)
print('Context window stats:')
print(f"  Total: {len(ctx)} | Unique: {len(ctx_freq)}")
print('  Top 10:', ctx_freq.most_common(10))
# Word-only metrics (exclude tokens without Hebrew/Latin letters)
word_only = [t for t in all_tokens if re.search(r'[A-Za-z\u0590-\u05FF]', t)]
word_only_freq = Counter(word_only)
word_only_vocab = set(word_only)
print('\nWord-only Lexical Stats:')
print(f"  Vocab size: {len(word_only_vocab):,}")
print(f"  Total tokens: {len(word_only):,}")
print(f"  TTR: {len(word_only_vocab)/len(word_only):.4f}")
hapax_w = sum(1 for v in word_only_freq.values() if v==1)
dis_w = sum(1 for v in word_only_freq.values() if v==2)
print(f"  Hapax legomena: {hapax_w} ({hapax_w/len(word_only_vocab)*100:.1f}% of vocab)")
print(f"  Dis legomena: {dis_w}")
print('  Top 10 words:', word_only_freq.most_common(10))


LEXICAL DIVERSITY
Vocabulary size: 15,107
Total tokens: 83,844
Type-Token Ratio (TTR): 0.1802
Hapax legomena: 8594 (56.9% of vocab)
Dis legomena: 2430
Top 10 tokens:
[(',', 3733), ('.', 3690), ('את', 2295), ('לא', 1296), ('הוא', 1107), ('היא', 1007), ('על', 918), ('של', 756), ('אם', 659), ('–', 558)]
Context window stats:
  Total: 23955 | Unique: 7095
  Top 10: [(',', 1424), ('הוא', 841), ('היא', 742), ('.', 557), ('לא', 456), ('הם', 420), ('על', 345), ('את', 301), ('עם', 243), ('כדי', 236)]

Word-only Lexical Stats:
  Vocab size: 15,089
  Total tokens: 74,883
  TTR: 0.2015
  Hapax legomena: 8588 (56.9% of vocab)
  Dis legomena: 2430
  Top 10 words: [('את', 2295), ('לא', 1296), ('הוא', 1107), ('היא', 1007), ('על', 918), ('של', 756), ('אם', 659), ('הם', 521), ('אחרי', 518), ('היה', 510)]


## 10. Structural Complexity

In [79]:

print('='*70)
print('STRUCTURAL COMPLEXITY')
print('='*70)
subclause_markers = ['ש','כי','אם','כאשר','למרות','בגלל','עד','לפני','אחרי']

def count_markers(tokens):
    toks = tokens if isinstance(tokens, list) else str(tokens).split()
    return sum(1 for t in toks for m in subclause_markers if t.startswith(m))

df['subclause_count'] = df['tokens'].apply(count_markers)
df['punct_count'] = df['sentence'].str.count(r'[.,!?;:\-–]')
print('Subclause count mean:', df['subclause_count'].mean())
print('Punctuation mean:', df['punct_count'].mean())
print('By label (punct mean):')
print(df.groupby('label')['punct_count'].mean())


STRUCTURAL COMPLEXITY
Subclause count mean: 1.9114583333333333
Punctuation mean: 1.7816666666666667
By label (punct mean):
label
0    1.728750
1    1.834583
Name: punct_count, dtype: float64


## 11. Morphological Variance (Hebrew-Specific)

In [80]:

print('='*70)
print('MORPHOLOGICAL VARIANCE')
print('='*70)
variants = df.groupby('base_pie')['pie_span'].nunique().sort_values(ascending=False)
print('Top 10 idioms by variant count:')
print(variants.head(10))

prefixes = ('ה','ו','ל','כ','ב','מ')
df['prefixed_token_present'] = df['tokens'].apply(lambda toks: any(t.startswith(prefixes) for t in toks))
print(f"Sentences with prefixed tokens: {df['prefixed_token_present'].sum()} ({df['prefixed_token_present'].mean()*100:.2f}%)")


MORPHOLOGICAL VARIANCE
Top 10 idioms by variant count:
base_pie
שם רגליים         35
שבר את הלב        32
פתח דלתות         29
סגר חשבון         28
הוריד פרופיל      23
יצא מהקופסה       22
קיבל סטירה        18
שם עליו פס        18
חתך פינה          18
ירד לו האסימון    17
Name: pie_span, dtype: int64
Sentences with prefixed tokens: 4800 (100.00%)


## 12. IOB Tag Validation

In [85]:

print('='*70)
print('IOB VALIDATION')
print('='*70)
valid_tags={'O','B-IDIOM','I-IDIOM'}
seq_errors=0
for _, row in df.iterrows():
    tags=row['iob_tags']
    if any(t not in valid_tags for t in tags):
        seq_errors+=1
        continue
    if len(tags)!=len(row['tokens']):
        seq_errors+=1
        continue
    for i,t in enumerate(tags):
        if t=='I-IDIOM' and (i==0 or tags[i-1] not in ('B-IDIOM','I-IDIOM')):
            seq_errors+=1
            break
print(f"IOB sequence/validity errors: {seq_errors}")


IOB VALIDATION
IOB sequence/validity errors: 0


## 13. Sentence Type Analysis

In [82]:

print('='*70)
print('SENTENCE TYPE ANALYSIS')
print('='*70)

def classify(text):
    text=str(text).strip()
    if text.endswith('?'): return 'Question'
    if text.endswith('!'): return 'Exclamatory'
    return 'Declarative'

df['sentence_type']=df['sentence'].apply(classify)
print(df['sentence_type'].value_counts())
print('By label:')
print(pd.crosstab(df['sentence_type'], df['label']))


SENTENCE TYPE ANALYSIS
sentence_type
Declarative    4549
Question        239
Exclamatory      12
Name: count, dtype: int64
By label:
label             0     1
sentence_type            
Declarative    2294  2255
Exclamatory       5     7
Question        101   138


## 14. Summary & Validation Report

In [83]:

print('='*70)
print('SUMMARY & VALIDATION REPORT')
print('='*70)
for k,v in validation_results.items():
    print(f"{k}:", '✅' if v else '❌')
print('All checks complete.')


SUMMARY & VALIDATION REPORT
Missing values: ✅
Duplicates: ✅
ID sequence: ✅
Label consistency: ✅
Encoding: ✅
Hebrew text: ✅
All checks complete.
